In [2]:
def recommended_batch_size(memory_use, max_tokens): # on the Alienware
    from math import log2
    # derived from GIST-Embedding-v0 runs
    gist_difficulty = 0.41 * 512
    difficulty = memory_use * max_tokens

    estimation = gist_difficulty / difficulty * 64
    return 2**int(log2(estimation))

recommended_batch_size(1.5, 512) # 64

usage: train.py [-h] [--model MODEL] [--data-path DATA_PATH]
                [--output-path OUTPUT_PATH] [--test-frac TEST_FRAC]
                [--seed SEED] [--vectorizer VECTORIZER] [--timer]
                [--embedding-model EMBEDDING_MODEL] [--chunk-size CHUNK_SIZE]
                [--samples-per-label SAMPLES_PER_LABEL]
                [--batch-size BATCH_SIZE]

options:
  -h, --help            show this help message and exit
  --model MODEL         Type of model to train
  --data-path DATA_PATH
                        Path to data
  --output-path OUTPUT_PATH
                        Path to save model
  --test-frac TEST_FRAC
                        Fraction of data to use for testing
  --seed SEED           Random seed
  --vectorizer VECTORIZER
                        Vectorizer to use
  --timer               Time the training process
  --embedding-model EMBEDDING_MODEL
                        Embedding model to use. Only used if --model is
                        EMBEDDINGS or 

In [2]:
!python preprocess.py \
    --use-default-paths \
    --only-irrelevant \
    --limit-irrelevant 50000




Using default paths:
Irrelevant: ../../data/unprocessed/irrelevant/
Spreadsheet: ../../data/unprocessed/spreadsheet.json
Synopses: ../../data/unprocessed/synopses/
Studies: ../../data/unprocessed/studies.json



Cleaning irrelevant data.
Loading ~10000 files...
 86%|███████████████████████████████▋     | 8556/10000 [00:13<00:02, 612.00it/s]

8556 irrelevant articles loaded. 
Removing corrupt files...
Removed 0 broken JSON files.
Saving irrelevant data...
Irrelevant data saved.


In [3]:
!python train.py \
    --model CuML \
    --data-path=../../data/level-0.5/data.json \
    --output-path=./models/level-1/cuML_classifier.pkl \
    --seed 1347 \
    --timer





Loading data...

/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 254, in ensure_initialized
    self.cuInit(0)
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 327, in safe_cuda_api_call
    self._check_ctypes_error(fname, retcode)
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 395, in _check_ctypes_error
    raise CudaAPIError(retcode, msg)
numba.cuda.cudadrv.driver.CudaAPIError: [100] Call to cuInit results in CUDA_ERROR_NO_DEVICE

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 292, in __getattr__
    self.ensure_init

In [3]:
!python predict.py \
    --model=CuML \
    --model-path=./models/level-1/cuML_classifier.pkl \
    --data-path=../../data/level-0.5/irrelevant.json \
    --output-path=../../results/level-1.5 \
    --level 1 \
    --save-top 1000 \
    --timer






Loading model from ./models/level-1/cuML_classifier.pkl...
Model loaded.

Loading data...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8556 entries, 0 to 8555
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           8556 non-null   object
 1   text          8556 non-null   object
 2   relevance     8556 non-null   object
 3   multiclasses  8556 non-null   object
dtypes: object(4)
memory usage: 267.5+ KB

Data loaded.

Generating classification predictions...

Classifying 8556 files.

Estimated time:  0.0  minutes  53  seconds



Testing time on 8556 articles:  64.36612033843994  seconds

Files processed per second: 132.92707335803632
Evaluating classifier...

prediction
irrelevant    8526
relevant        30
Name: count, dtype: int64
Accuracy:  0.9964936886395512
Precision:  0.5
Recall:  0.4982468443197756
F1:  0.49912188268352653
Confusion matrix: 
 [[8526   30]
 [   0    0]]


Potential new Conserv

In [1]:
!python predict.py \
    --model=FastFit \
    --model-path=./models/level-2/avsolatorio/GIST-large-Embedding-v0 \
    --data-path=../../data/level-1.5/potential.json \
    --output-path=../../results/level-2.5/ \
    --save-top 50 \
    --batch-size 64 \
    --level 2 \
    --timer






************ RUNNING LEVEL 2 PREDICTIONS WITH EMBEDDING MODEL ************

Model: ['avsolatorio', 'GIST-large-Embedding-v0']


Initialising classifier...
Loading model from ./models/level-2/avsolatorio/GIST-large-Embedding-v0 ...
Model loaded.
Tokenizer loaded with max_length: 512
Building classifier pipeline...
NOTE: Ignore the below warning about 'FastFi' not being supported for text-classification; it's not necessary.
The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenc

In [1]:
!python train.py \
    --model FastFit \
    --data-path=../../data/level-0.5/data.json \
    --embedding-model=avsolatorio/GIST-Embedding-v0 \
    --output-path=./models/level-2/ \
    --seed 1347 \
    --chunk-size 512 \
    --batch-size 32 \
    --timer







************ TRAINING EMBEDDING MODEL ************

avsolatorio/GIST-Embedding-v0


Loading data...
Data loaded. 
Splitting data...
Data split.

Chunking data...
100%|█████████████████████████████████████| 3078/3078 [00:01<00:00, 2808.82it/s]
Data chunked.

Downsampling data...
Data downsampled.

Converting data to FastFit format...

Train data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses'],
    num_rows: 2122614
})

Test data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses', '__index_level_0__'],
    num_rows: 200
})

Initialising FastFit trainer...
[WARNING  | fastfit.train      ]: Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
tokenizer_config.json: 100%|███████████████| 1.24k/1.24k [00:00<00:00, 4.63MB/s]
vocab.txt: 100%|█████████████████████████████| 232k/232k [00:00<00:00, 1.39MB/s]
tokenizer.json: 100%|████████████████████████| 711k/711k [00:00<00:00, 2.34MB/s]
co

In [2]:
!python train.py \
    --model FastFit \
    --data-path=../../data/level-0.5/data.json \
    --embedding-model=avsolatorio/GIST-small-Embedding-v0 \
    --output-path=./models/level-2/ \
    --seed 1347 \
    --chunk-size 512 \
    --batch-size 64 \
    --samples-per-label 1000 \
    --timer







************ TRAINING EMBEDDING MODEL ************

avsolatorio/GIST-small-Embedding-v0


Loading data...
Data loaded. 
Splitting data...
Data split.

Chunking data...
100%|█████████████████████████████████████| 3078/3078 [00:01<00:00, 2813.99it/s]
Data chunked.

Downsampling data...
Data downsampled.

Converting data to FastFit format...

Train data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses'],
    num_rows: 2000
})

Test data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses', '__index_level_0__'],
    num_rows: 200
})

Initialising FastFit trainer...
[WARNING  | fastfit.train      ]: Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset to infer max length for both query and document: 10

In [3]:
!python train.py \
    --model FastFit \
    --data-path=../../data/level-0.5/data.json \
    --embedding-model=avsolatorio/GIST-large-Embedding-v0 \
    --output-path=./models/level-2/ \
    --chunk-size 512 \
    --batch-size 32 \
    --samples-per-label 1000 \
    --timer






************ TRAINING EMBEDDING MODEL ************

avsolatorio/GIST-large-Embedding-v0


Loading data...
Data loaded. 
Splitting data...
Data split.

Chunking data...
100%|█████████████████████████████████████| 3078/3078 [00:00<00:00, 3219.69it/s]
Data chunked.

Downsampling data...
Data downsampled.

Converting data to FastFit format...

Train data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses'],
    num_rows: 2000
})

Test data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses', '__index_level_0__'],
    num_rows: 200
})

Initialising FastFit trainer...
[WARNING  | fastfit.train      ]: Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset to infer max length for both query and document: 10